### Libraries import

In [32]:
from dotenv import load_dotenv
import os
import requests
import urllib3
import pandas as pd
from tqdm import tqdm
import time
import random
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
pd.set_option('display.max_columns', None)

In [33]:
from sqlalchemy import Text, Integer, BigInteger, Float, Boolean
from sqlalchemy import create_engine
from sqlalchemy.dialects.postgresql import JSONB

### Load evn variables

In [34]:
load_dotenv()

True

### Strava API setup

In [35]:
auth_url = 'https://www.strava.com/oauth/token'
activites_url = 'https://www.strava.com/api/v3/athlete/activities'

payload = {
    'client_id': os.getenv('CLIENT_ID'),
    'client_secret': os.getenv('CLIENT_SECRET'),
    'refresh_token': os.getenv('REFRESH_TOKEN'),
    'grant_type': 'refresh_token',
    'f': 'json'
}

### SQL setup

In [36]:
engine = create_engine(os.getenv('DB_URI'))

### Requesing list of all activities

In [37]:
print('Requesting Token...\n')
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']

print('Access Token = {}\n'.format(access_token))
header = {'Authorization': 'Bearer ' + access_token}

# The first loop, request_page_number will be set to one, so it requests the first page. Increment this number after
# each request, so the next time we request the second page, then third, and so on...
request_page_num = 1
all_activities = []

while True:
    param = {'per_page': 200, 'page': request_page_num}
    # initial request, where we request the first page of activities
    my_dataset = requests.get(activites_url, headers=header, params=param).json()

    # check the response to make sure it is not empty. If it is empty, that means there is no more data left. So if you have
    # 1000 activities, on the 6th request, where we request page 6, there would be no more data left, so we will break out of the loop
    if len(my_dataset) == 0:
        print('breaking out of while loop because the response is zero, which means there must be no more activities')
        break

    # if the all_activities list is already populated, that means we want to add additional data to it via extend.
    if all_activities:
        print('all_activities is populated')
        all_activities.extend(my_dataset)

    # if the all_activities is empty, this is the first time adding data so we just set it equal to my_dataset
    else:
        print('all_activities is NOT populated')
        all_activities = my_dataset

    request_page_num += 1

print(len(all_activities))
for count, activity in enumerate(all_activities):
    print(activity['name'])
    print(count)

all_activities_df = pd.json_normalize(all_activities, sep='_')

Requesting Token...

Access Token = 62885dc0d0678b8b2fadd0eb5bb460f19d28095d

all_activities is NOT populated
all_activities is populated
all_activities is populated
all_activities is populated
all_activities is populated
all_activities is populated
breaking out of while loop because the response is zero, which means there must be no more activities
1085
Lunch Ride
0
24km Race Practice Long Run🔩
1
Evening Ride
2
Afternoon Weight Training
3
Afternoon Ride
4
800m Repeats🚀
5
Evening Ride
6
Afternoon Weight Training
7
Afternoon Ride
8
Afternoon Walk
9
Afternoon Ride
10
24km Race Practice Long Run🪦
11
Coffee Race 2025☕️
12
9km Easy Run🌅
13
Afternoon Weight Training
14
Afternoon Ride
15
Rolling 300s🪦
16
Afternoon Weight Training
17
Progressive Run🪦
18
22km Long Run🗿
19
Afternoon Weight Training
20
Evening Ride
21
400m Repeats😮‍💨
22
9km Easy Run✨
23
Afternoon Weight Training
24
Afternoon Ride
25
Afternoon Ride
26
11km Long Run🗿
27
Evening Ride
28
Morning Ride
29
Afternoon Walk
30
Afternoon We

### Limiting size of data download

In [38]:
top_n = 100
all_activities_df = all_activities_df.iloc[:top_n] 

In [39]:
all_activities_df.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_watts,device_watts,kilojoules,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,athlete_id,athlete_resource_state,map_id,map_summary_polyline,map_resource_state,average_cadence,max_watts,weighted_average_watts
0,2,Lunch Ride,79588.5,11082,14430,202.0,Ride,Ride,NaN,15729456618,2025-09-07T09:45:26Z,2025-09-07T11:45:26Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,44,13,0,1,0,False,False,False,False,followers_only,False,b12572672,"[51.108316, 17.123345]","[51.107901, 17.123794]",7.182,11.18,23.0,183.2,False,2029.7,True,129.0,148.0,False,True,158.4,115.4,1.680192e+10,16801924720,garmin_ping_477733386080,False,20,0,False,53.0,81055898,1,a15729456618,kv}vHw~mgBqMlTwFbHa@pA`LzVaAzNy@`Pf@bZFtYrAfPC...,2,NaN,NaN,NaN
1,2,24km Race Practice Long Run🔩,24120.3,8004,8085,56.0,Run,Run,2.0,15716821076,2025-09-06T10:41:12Z,2025-09-06T12:41:12Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,17,13,2,2,0,False,False,False,False,everyone,False,g23642256,"[51.107164, 17.123723]","[51.106689, 17.123415]",3.014,4.28,NaN,369.3,True,2956.2,True,154.9,173.0,False,True,123.2,111.4,1.678855e+10,16788554675,garmin_ping_477379370380,False,10,0,False,165.0,81055898,1,a15716821076,{`|vHsrogBz@fBdAbCDX?t@Ob@aBbCmCfD{JnKuChDuBxC...,2,84.8,581.0,375.0
2,2,Evening Ride,16823.7,3683,6122,47.0,Ride,Ride,NaN,15708639235,2025-09-05T16:31:17Z,2025-09-05T18:31:17Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,5,10,0,4,0,False,False,False,False,followers_only,False,b12572672,"[51.107755, 17.123295]","[51.107903, 17.12546]",4.568,12.62,20.0,95.9,False,353.1,True,101.0,145.0,False,True,126.2,116.2,1.677985e+10,16779848219,garmin_ping_477117769680,False,0,1,False,9.0,81055898,1,a15708639235,kt}vHkangBkAhBKh@_@x@_CnDa@v@}@|AiA`B}@|A{B~Cw...,2,NaN,NaN,NaN
3,2,Afternoon Weight Training,0.0,3713,3713,0.0,Workout,WeightTraining,NaN,15705468575,2025-09-05T12:00:34Z,2025-09-05T14:00:34Z,(GMT+02:00) Africa/Blantyre,7200.0,None,None,None,0,8,0,1,0,True,False,False,False,followers_only,False,None,[],[],0.000,0.00,NaN,NaN,NaN,NaN,True,99.6,142.0,False,True,0.0,0.0,1.677651e+10,16776511359,garmin_ping_477033364177,False,0,0,False,8.0,81055898,1,a15705468575,,2,NaN,NaN,NaN
4,2,Afternoon Ride,13045.3,1871,6563,44.0,Ride,Ride,10.0,15705659558,2025-09-05T11:32:00Z,2025-09-05T13:32:00Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,9,12,0,1,0,False,True,False,False,followers_only,False,b12572672,"[51.1085, 17.123504]","[51.107656, 17.125015]",6.972,10.50,25.0,181.2,False,339.1,True,134.1,152.0,False,True,129.0,115.4,1.677671e+10,16776713564,garmin_ping_477039270074,False,4,0,False,13.0,81055898,1,a15705659558,yv}vH_~mgBmFfJeAvAm@fAgC~DqAxAcAtAu@z@g@v@ELDX...,2,NaN,NaN,NaN


### Requesing list of all activities with details

In [40]:
print('Requesting Token...\n')
res = requests.post(auth_url, data=payload, verify=False) # auth_url & payload referenced in code already above
access_token = res.json()['access_token']
header = {'Authorization': 'Bearer ' + access_token}
print('Access Token = {}\n'.format(access_token))

all_activities_details = []
count = 0

print('Calculating time to get all activities details....\n')
print(all_activities_df.shape[0] * 8 / 60, ' minutes to obtain data')

print('Getting details of each activity')
for index, row in tqdm(all_activities_df.iterrows(), total=all_activities_df.shape[0]):
    get_activity_url = 'https://www.strava.com/api/v3/activities/{}'.format(row['id'])
    try:
        activity_details = requests.get(get_activity_url, headers=header).json() 
        all_activities_details.append(activity_details)
        time.sleep(random.randint(7, 9))
    except requests.exception.RequestException as e:
        raise SystemExit(e)

all_activities_details_df = pd.json_normalize(all_activities_details, sep='_')

Requesting Token...

Access Token = 62885dc0d0678b8b2fadd0eb5bb460f19d28095d

Calculating time to get all activities details....

13.333333333333334  minutes to obtain data
Getting details of each activity


100%|██████████| 100/100 [14:56<00:00,  8.96s/it]


In [41]:
all_activities_details_df

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_watts,device_watts,kilojoules,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,description,calories,perceived_exertion,prefer_perceived_exertion,segment_efforts,splits_metric,splits_standard,laps,stats_visibility,hide_from_home,device_name,embed_token,available_zones,athlete_id,athlete_resource_state,map_id,map_polyline,map_resource_state,map_summary_polyline,gear_primary,gear_name,gear_nickname,gear_resource_state,gear_retired,gear_distance,gear_converted_distance,photos_primary,photos_count,average_cadence,max_watts,weighted_average_watts,best_efforts,similar_activities_effort_count,similar_activities_average_speed,similar_activities_min_average_speed,similar_activities_mid_average_speed,similar_activities_max_average_speed,similar_activities_pr_rank,similar_activities_frequency_milestone,similar_activities_trend_speeds,similar_activities_trend_current_activity_index,similar_activities_trend_min_speed,similar_activities_trend_mid_speed,similar_activities_trend_max_speed,similar_activities_trend_direction,similar_activities_resource_state,photos_primary_unique_id,photos_primary_urls_600,photos_primary_urls_100,photos_primary_source,photos_primary_media_type,photos_use_primary_photo,private_note
0,3,Lunch Ride,79588.5,11082,14430,202.0,Ride,Ride,NaN,15729456618,2025-09-07T09:45:26Z,2025-09-07T11:45:26Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,44,13,0,1,0,False,False,False,False,followers_only,False,b12572672,"[51.108316, 17.123345]","[51.107901, 17.123794]",7.182,11.18,23.0,183.2,False,2029.7,True,129.0,148.0,False,True,158.4,115.4,1.680192e+10,16801924720,garmin_ping_477733386080,False,20,0,False,53.0,Nogi nie współpracowały po wczorajszym longu🪦,1388.0,None,None,"[{'id': 3399769248862229720, 'resource_state':...","[{'distance': 1003.5, 'elapsed_time': 137, 'el...","[{'distance': 1614.2, 'elapsed_time': 235, 'el...","[{'id': 56016268643, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin Edge 840,246e2c4b121e976191c2c4d98a055e939da834c6,"[heartrate, power]",81055898,1,a15729456618,}a}vH{kogBILs@x@WP_@PGJKp@O|BWxAM`@c@|@o@~@KZ?...,3,kv}vHw~mgBqMlTwFbHa@pA`LzVaAzNy@`Pf@bZFtYrAfPC...,False,Cube Nuroad Pro,Cube Nuroad Pro,2.0,False,3734608.0,3734.6,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,24km Race Practice Long Run🔩,24120.3,8004,8085,56.0,Run,Run,2.0,15716821076,2025-09-06T10:41:12Z,2025-09-06T12:41:12Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,17,13,2,2,0,False,False,False,False,everyone,False,g23642256,"[51.107164, 17.123723]","[51.106689, 17.123415]",3.014,4.28,NaN,369.3,True,2956.2,True,154.9,173.0,False,True,123.2,111.4,1.678855e+10,16788554675,garmin_ping_477379370380,False,10,0,False,165.0,24km Race Practice Long Run with Runna ✅\n\nDo...,1857.0,None,None,"[{'id': 3399398286614446506, 'resource_state':...","[{'distance': 1000.0, 'elapsed_time': 342, 'el...","[{'distance': 1609.4, 'elapsed_time': 553, 'el...","[{'id': 55964572174, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin Forerunner 970,03edfc9cb2149366844c599b974197a1867550de,"[heartrate, pace, power]",81055898,1,a15716821076,wz|vHgnogBPx@Rd@JFHC^c@DCTLV\DBNCr@u@NS`AcAz@k...,3,{`|vHsrogBz@fBdAbCDX?t@Ob@aBbCmCfD{JnKuChDuBxC...,False,Adidas EVO SL,None,2.0,False,146967.0,147.0,NaN,0,84.8,581.0,375.0,"[{'id': 65928416490, 'resource_state': 2,

### Requesing kudos related to activities

In [42]:
print('Requesting Token...\n')
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
header = {'Authorization': 'Bearer ' + access_token}
print('Access Token = {}\n'.format(access_token))

all_activities_kudos = []
count = 0

print('Calculating time to get all activities details....\n')
print(all_activities_df.shape[0] * 8 / 60, ' minutes to obtain data')

print('Getting details of each activity')
for index, row in tqdm(all_activities_df.iterrows(), total=all_activities_df.shape[0]):
    get_kudos_url = 'https://www.strava.com/api/v3/activities/{}/kudos'.format(row['id'])
    try:
        activity_kudos = requests.get(get_kudos_url, headers=header).json()
        if len(activity_kudos) > 0:
            for kudos in activity_kudos:
                kudos['activity_id'] = row['id']
                all_activities_kudos.extend(activity_kudos)
        time.sleep(random.randint(7, 9))
    except requests.exception.RequestException as e:
        raise SystemExit(e)
    
all_activities_kudos_df = pd.json_normalize(all_activities_kudos, sep='_')

Requesting Token...

Access Token = 62885dc0d0678b8b2fadd0eb5bb460f19d28095d

Calculating time to get all activities details....

13.333333333333334  minutes to obtain data
Getting details of each activity


100%|██████████| 100/100 [14:28<00:00,  8.69s/it]


In [43]:
all_activities_kudos_df

,resource_state,firstname,lastname,activity_id
0,2,Kacper,G.,15729456618
1,2,Jan,K.,15729456618
2,2,Jacek,S.,15729456618
3,2,jakub,B.,15729456618
4,2,Ola,Ł.,15729456618
...,...,...,...,...
13706,2,Wiesława,C.,14673647608
13707,2,Łukasz,C.,14673647608
13708,2,Karolina,C.,14673647608
13709,2,Krzysztof,S.,14673647608


### Load data to PostgreSQL

In [44]:
with engine.begin() as conn:
    conn.exec_driver_sql("CREATE SCHEMA IF NOT EXISTS bronze;")

activities_dtype_map = {
    "resource_state": Integer,
    "name": Text,
    "distance": Float,
    "moving_time": Integer,
    "elapsed_time": Integer,
    "total_elevation_gain": Float,
    "type": Text,
    "sport_type": Text,
    "workout_type": Float,
    "id": BigInteger,
    "start_date": Text,
    "start_date_local": Text,
    "timezone": Text,
    "utc_offset": Float,
    "location_city": Text,
    "location_state": Text,
    "location_country": Text,
    "achievement_count": Integer,
    "kudos_count": Integer,
    "comment_count": Integer,
    "athlete_count": Integer,
    "photo_count": Integer,
    "trainer": Boolean,
    "commute": Boolean,
    "manual": Boolean,
    "private": Boolean,
    "visibility": Text,
    "flagged": Boolean,
    "gear_id": Text,
    "start_latlng": JSONB,
    "end_latlng": JSONB,
    "average_speed": Float,
    "max_speed": Float,
    "average_cadence": Float,
    "average_watts": Float,
    "max_watts": Float,
    "weighted_average_watts": Float,
    "device_watts": Boolean,
    "kilojoules": Float,
    "has_heartrate": Boolean,
    "average_heartrate": Float,
    "max_heartrate": Float,
    "heartrate_opt_out": Boolean,
    "display_hide_heartrate_option": Boolean,
    "elev_high": Float,
    "elev_low": Float,
    "upload_id": BigInteger,
    "upload_id_str": Text,
    "external_id": Text,
    "from_accepted_tag": Boolean,
    "pr_count": Integer,
    "total_photo_count": Integer,
    "has_kudoed": Boolean,
    "suffer_score": Float,
    "athlete_id": BigInteger,
    "athlete_resource_state": Integer,
    "map_id": Text,
    "map_summary_polyline": Text,
    "map_resource_state": Integer,
    "average_temp": Float,
}

activities_details_dtype_map = {
    "resource_state": Integer,
    "name": Text,
    "distance": Float,
    "moving_time": Integer,
    "elapsed_time": Integer,
    "total_elevation_gain": Float,
    "type": Text,
    "sport_type": Text,
    "workout_type": Float,
    "id": BigInteger,
    "start_date": Text,
    "start_date_local": Text,
    "timezone": Text,
    "utc_offset": Float,
    "location_city": Text,
    "location_state": Text,
    "location_country": Text,
    "achievement_count": Integer,
    "kudos_count": Integer,
    "comment_count": Integer,
    "athlete_count": Integer,
    "photo_count": Integer,
    "trainer": Boolean,
    "commute": Boolean,
    "manual": Boolean,
    "private": Boolean,
    "visibility": Text,
    "flagged": Boolean,
    "gear_id": Text,
    "start_latlng": JSONB,
    "end_latlng": JSONB,
    "average_speed": Float,
    "max_speed": Float,
    "average_cadence": Float,
    "average_watts": Float,
    "max_watts": Float,
    "weighted_average_watts": Float,
    "device_watts": Boolean,
    "kilojoules": Float,
    "has_heartrate": Boolean,
    "average_heartrate": Float,
    "max_heartrate": Float,
    "heartrate_opt_out": Boolean,
    "display_hide_heartrate_option": Boolean,
    "elev_high": Float,
    "elev_low": Float,
    "upload_id": BigInteger,
    "upload_id_str": Text,
    "external_id": Text,
    "from_accepted_tag": Boolean,
    "pr_count": Integer,
    "total_photo_count": Integer,
    "has_kudoed": Boolean,
    "suffer_score": Float,
    "description": Text,
    "calories": Float,
    "perceived_exertion": Text,
    "prefer_perceived_exertion": Text,
    "segment_efforts": JSONB,
    "splits_metric": JSONB,
    "splits_standard": JSONB,
    "laps": JSONB,
    "best_efforts": JSONB,
    "stats_visibility": JSONB,
    "hide_from_home": Boolean,
    "device_name": Text,
    "embed_token": Text,
    "available_zones": JSONB,
    "athlete_id": BigInteger,
    "athlete_resource_state": Integer,
    "map_id": Text,
    "map_polyline": Text,
    "map_resource_state": Integer,
    "map_summary_polyline": Text,
    "gear_primary": Boolean,
    "gear_name": Text,
    "gear_nickname": Text,
    "gear_resource_state": Float,
    "gear_retired": Boolean,
    "gear_distance": Float,
    "gear_converted_distance": Float,
    "photos_primary": JSONB,
    "photos_count": Integer,
    "similar_activities_effort_count": Float,
    "similar_activities_average_speed": Float,
    "similar_activities_min_average_speed": Float,
    "similar_activities_mid_average_speed": Float,
    "similar_activities_max_average_speed": Float,
    "similar_activities_pr_rank": Float,
    "similar_activities_frequency_milestone": Float,
    "similar_activities_trend_speeds": JSONB,
    "similar_activities_trend_current_activity_index": Float,
    "similar_activities_trend_min_speed": Float,
    "similar_activities_trend_mid_speed": Float,
    "similar_activities_trend_max_speed": Float,
    "similar_activities_trend_direction": Float,
    "similar_activities_resource_state": Float,
    "average_temp": Float,
}

activities_kudos_dtype_map = {
    "resource_state": Integer,
    "firstname": Text,
    "lastname": Text,
    "activity_id": BigInteger,
}


all_activities_df.to_sql(
    name="activities",
    schema="bronze",
    con=engine,
    if_exists="replace",
    index=False,
    dtype=activities_dtype_map,
    method="multi",
    chunksize=1000
)

all_activities_details_df.to_sql(
    name="activities_details",
    schema="bronze",
    con=engine,
    if_exists="replace",
    index=False,
    dtype=activities_details_dtype_map,
    method="multi",
    chunksize=1000
)

all_activities_kudos_df.to_sql(
    name="kudos",
    schema="bronze",
    con=engine,
    if_exists="replace",
    index=False,
    dtype=activities_kudos_dtype_map,
    method="multi",
    chunksize=1000
)

-14